## Импорт библиотек 📦

In [7]:
import pandas as pd

import seaborn as sns

import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import numpy as np

from copy import deepcopy

from catboost import Pool, cv
from catboost import CatBoostClassifier
from catboost import CatBoostRegressor

pd.set_option('display.max_columns', None)
%matplotlib inline

# Работа с данными 📩🔬🧮

__Импорт данных 📥__

In [8]:
df_issues_train = pd.read_csv("content/train_issues.csv")
df_comment_train = pd.read_csv("content/train_comments.csv")

df_issues_test = pd.read_csv("content/test_issues.csv")
df_comment_test = pd.read_csv("content/test_comments.csv")

df_emp = pd.read_csv("content/employees.csv")

df_sample_solution = pd.read_csv("content/sample_solution.csv")

__Быстрый взгляд и общий сбор 🔬__

In [9]:
df_train = pd.merge(df_issues_train, df_emp, \
                    left_on="assignee_id", right_on="id", how='inner',suffixes = ('_issue','_employee'))

df_train = pd.merge(df_train, df_emp, \
                    left_on="creator_id", right_on="id", how='inner',suffixes = ('','_master'))

df_train = pd.merge(df_train, df_comment_train.groupby(['issue_id'])['text'].apply(list).reset_index(), \
                   left_on="id_issue", right_on="issue_id", how='left',suffixes = ('','_comment'))
df_train

,id_issue,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,id_employee,active,full_name,position,hiring_type,payment_type,salary_calculation_type,english_level,passport,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,id,active_master,full_name_master,position_master,hiring_type_master,payment_type_master,salary_calculation_type_master,english_level_master,passport_master,is_nda_signed_master,is_labor_contract_signed_master,is_added_to_internal_chats_master,is_added_one_to_one_master,issue_id,text
0,819952,2019-10-01 05:57:18.000,SM-10678,"UI тесты по заказу ""Добро КейДжи""",5,93,93,1800,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN
1,819949,2019-10-01 05:59:45.000,SM-10679,"UI тесты раздела ""Профиль""",5,93,93,7200,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,819949.0,[Приверила и приняла MR\n\n ]
2,819947,2019-10-01 06:00:38.000,SM-10680,"UI тесты раздела ""Личный счет""",5,93,93,14400,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN
3,819943,2019-10-01 06:02:49.000,SM-10682,"UI тесты раздела ""Новости""",5,93,93,900,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN
4,819941,2019-10-01 06:03:26.000,SM-10683,"UI тесты раздела ""Зоны скидок и доплат""",5,93,93,900,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,819941.0,[Приняла mR]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9584,688489,2020-01-08 13:42:46.000,XXO-191,Created Design for a additional content webpages,33,246,246,14400,246,1,Jacklyn Cullum,Web дизайнер,staff,hourly,gross,NaN,1,1,1,1,1,246,1,Jacklyn Cullum,Web дизайнер,staff,hourly,gross,NaN,1,1,1,1,1,688489.0,[clients are happy with a design]
9585,670934,2020-06-02 10:08:14.000,LJ-1,Set up the neo4j by using a dataset from Slack,56,2,2,16200,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,670934.0,[I have read the neo4j documentation and also ...
9586,670932,2020-06-05 09:52:12.000,LJ-3,Set up a graph in neo4j by GitLab dataset,56,2,2,3600,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,NaN,NaN
9587,670931,2020-06-05 13:46:38.000,LJ-4,Parse a dataset from teacherly slack,56,2,2,6300,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,NaN,NaN


In [10]:
df_test = pd.merge(df_issues_test, df_emp, \
                    left_on="assignee_id", right_on="id", how='inner',suffixes = ('_issue','_employee'))

df_test = pd.merge(df_test, df_emp, \
                    left_on="creator_id", right_on="id", how='inner',suffixes = ('','_master'))

df_test = pd.merge(df_test, df_comment_test.groupby(['issue_id'])['text'].apply(list).reset_index(), \
                   left_on="id_issue", right_on="issue_id", how='left',suffixes = ('','_comment'))
df_test

,id_issue,created,key,summary,project_id,assignee_id,creator_id,id_employee,active,full_name,position,hiring_type,payment_type,salary_calculation_type,english_level,passport,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,id,active_master,full_name_master,position_master,hiring_type_master,payment_type_master,salary_calculation_type_master,english_level_master,passport_master,is_nda_signed_master,is_labor_contract_signed_master,is_added_to_internal_chats_master,is_added_one_to_one_master,issue_id,text
0,675975,2020-01-15 09:52:57.000,SM-11140,Выгрузить все mp3 файлы звонков на диск,5,112,112,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,675975.0,[Ресерч по rsync:\n\n[https://losst.ru/rsync-p...
1,675972,2020-01-15 09:54:35.000,SM-11141,Удалить файлы mp3 звонков с сервера до 01.01.2019,5,112,112,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,675972.0,[Задача приостановлена в связи с заморозкой ра...
2,675857,2020-05-04 09:47:24.000,SM-11186,настройка DNS на cloudflare,5,112,112,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,675857.0,[Создал учетку в клауд флейр для дальнешего пе...
3,675849,2020-05-12 08:53:22.000,SM-11189,Выяснить действительно ли причина в поломанной...,5,112,112,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,675849.0,[Посмотрел видео по репликации\r\n\r\n[https:/...
4,675845,2020-05-13 12:02:28.000,SM-11191,Разобраться с проблемой репликации,5,112,112,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,675845.0,[Прочитал доку [https://docs.google.com/docume...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,1990701,2021-06-22 17:23:10.572,FPY-71,Добавить базовую аутентификацию для АПИ к камерам,29,191,191,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,NaN,NaN
1066,2105184,2021-07-02 15:00:08.924,FPY-79,[back] Настроить отправку писем после регистрации,29,191,191,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,NaN,NaN
1067,2170252,2021-07-08 13:05:34.082,FPY-82,Обновлять статус камеры на неактивный при отсу...,29,191,191,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,2170252.0,[В настройки проекта в constance вынес конфиг ...
1068,2176348,2021-07-15 12:41:44.073,FPY-90,Проверить почему скрипт деактивирует активную ...,29,191,191,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,2176348.0,"[Причину определили, во время демки камеры не ..."


In [11]:
df_train['created'] = df_train['created'].apply(pd.to_datetime)
df_test['created'] = df_test['created'].apply(pd.to_datetime)

In [48]:
from functools import reduce

def temp_converter(comms):
    if type(comms) == type(0.1):
        return '-'
    else:
        return ' '.join(comms)
    
def thereis_converter(comms):
    if type(comms) == type(0.1):
        return 0
    else:
        return 1

def len_converter(comms):
    if type(comms) == type(0.1):
        return 0
    else:
        return len(comms)    
    
test_comments = list(map(temp_converter, list(df_test['text'])))
train_comments = list(map(temp_converter, list(df_train['text'])))

test_thereis_comment = list(map(thereis_converter, list(df_test['text'])))
train_thereis_comment = list(map(thereis_converter, list(df_train['text'])))

test_len_comment = list(map(len_converter, list(df_test['text'])))
train_len_comment = list(map(len_converter, list(df_train['text'])))

df_train['comment'], df_test['comment']  = train_comments, test_comments
df_train['any_comment'], df_test['any_comment']  = train_thereis_comment, test_thereis_comment
df_train['len_comment'], df_test['len_comment']  = train_len_comment , test_len_comment

df_train

,id_issue,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,id_employee,active,full_name,position,hiring_type,payment_type,salary_calculation_type,english_level,passport,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,id,active_master,full_name_master,position_master,hiring_type_master,payment_type_master,salary_calculation_type_master,english_level_master,passport_master,is_nda_signed_master,is_labor_contract_signed_master,is_added_to_internal_chats_master,is_added_one_to_one_master,issue_id,text,comment,any_comment,len_comment
0,819952,2019-10-01 05:57:18,SM-10678,"UI тесты по заказу ""Добро КейДжи""",5,93,93,1800,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN,-,0,0
1,819949,2019-10-01 05:59:45,SM-10679,"UI тесты раздела ""Профиль""",5,93,93,7200,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,819949.0,[Приверила и приняла MR\n\n ],Приверила и приняла MR\n\n,1,1
2,819947,2019-10-01 06:00:38,SM-10680,"UI тесты раздела ""Личный счет""",5,93,93,14400,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN,-,0,0
3,819943,2019-10-01 06:02:49,SM-10682,"UI тесты раздела ""Новости""",5,93,93,900,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN,-,0,0
4,819941,2019-10-01 06:03:26,SM-10683,"UI тесты раздела ""Зоны скидок и доплат""",5,93,93,900,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,819941.0,[Приняла mR],Приняла mR,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9584,688489,2020-01-08 13:42:46,XXO-191,Created Design for a additional content webpages,33,246,246,14400,246,1,Jacklyn Cullum,Web дизайнер,staff,hourly,gross,NaN,1,1,1,1,1,246,1,Jacklyn Cullum,Web дизайнер,staff,hourly,gross,NaN,1,1,1,1,1,688489.0,[clients are happy with a design],clients are happy with a design,1,1
9585,670934,2020-06-02 10:08:14,LJ-1,Set up the neo4j by using a dataset from Slack,56,2,2,16200,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,670934.0,[I have read the neo4j documentation and also ...,I have read the neo4j documentation and also I...,1,3
9586,670932,2020-06-05 09:52:12,LJ-3,Set up a graph in neo4j by GitLab dataset,56,2,2,3600,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,NaN,NaN,-,0,0
9587,670931,2020-06-05 13:46:38,LJ-4,Parse a dataset from teacherly slack,56,2,2,6300,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,NaN,NaN,-,0,0


In [51]:
train_lines = list(df_train['summary'])
with open('train_to_translate_text.txt', 'w', encoding="utf-8") as f:
    for line in train_lines:
        f.write(f"{line}\n")
        

test_lines = list(df_test['summary'])
with open('test_to_translate_text.txt', 'w', encoding="utf-8") as f:
    for line in test_lines:
        f.write(f"{line}\n")
        
        
test_comments = list(df_test['comment'])
with open('test_to_translate_comments.txt', 'w', encoding="utf-8") as f:
    for line in test_comments:
        f.write(f"{line}\n")

train_comments = list(df_train['comment'])
with open('train_to_translate_comments.txt', 'w', encoding="utf-8") as f:
    for line in train_comments:
        f.write(f"{line}\n")

❗❗❗ Здесь произошёл перевод

In [57]:
train_translated_summary = []
with open('train_translated_text.txt', encoding="utf-8") as f:
    for i in range(len(train_lines)):
        train_translated_summary += [f.readline().strip('\n').strip('\ufeff')]

test_translated_summary = []
with open('test_translated_text.txt', encoding="utf-8") as f:
    for i in range(len(test_lines)):
        test_translated_summary += [f.readline().strip('\n').strip('\ufeff')]
        
test_translated_comments = []
with open('test_translated_comments.txt', encoding="utf-8") as f:
    for i in range(len(test_lines)):
        test_translated_comments += [f.readline().strip('\n').strip('\ufeff')]
        
train_translated_comments = []
with open('train_translated_comments.txt', encoding="utf-8") as f:
    for i in range(len(train_lines)):
        train_translated_comments += [f.readline().strip('\n').strip('\ufeff')]

In [58]:
df_train['translated_summary'] = train_translated_summary
df_test['translated_summary'] = test_translated_summary
df_train['translated_comments'] = train_translated_comments
df_test['translated_comments'] = test_translated_comments

In [12]:
df_train.dtypes

id_issue                                      int64
created                              datetime64[ns]
key                                          object
summary                                      object
project_id                                    int64
assignee_id                                   int64
creator_id                                    int64
overall_worklogs                              int64
id_employee                                   int64
active                                        int64
full_name                                    object
position                                     object
hiring_type                                  object
payment_type                                 object
salary_calculation_type                      object
english_level                                object
passport                                      int64
is_nda_signed                                 int64
is_labor_contract_signed                      int64
is_added_to_

In [59]:
cat_need_columns = ['project_id',  # id проекта внутри команды
                    'assignee_id',  # id человека решающего
                    'creator_id',  # id человека проверяющего 
                    'active', # активность решающего
                    'position', # должность решающего
                    'hiring_type', # тип найма решающего
                    'payment_type', # как платят решающему
                    'salary_calculation_type', # как считается зарплата решающего
                    'english_level', # уровень английского у решающего
                    'passport', # паспорт решающего ????
                    'is_nda_signed', # подписан NDA решающим
                    'is_labor_contract_signed', # подписан рабочий контракт решающим
                    'is_added_to_internal_chats', # добавлен во внутренний чат решающий
                    'is_added_one_to_one', # добавлен один к одному решающий????
                    'active_master', # активность проверяющего
                    'position_master', # должность проверяющего
                    'hiring_type_master', # тип найма проверяющего
                    'payment_type_master', # как платят проверяющего
                    'salary_calculation_type_master', # как считается зарплата проверяющего
                    'english_level_master', # уровень английского у проверяющего
                    'passport_master', # паспорт проверяющего ????
                    'is_nda_signed_master', # подписан NDA проверяющим
                    'is_labor_contract_signed_master', # подписан рабочий контракт проверяющим
                    'is_added_to_internal_chats_master', # добавлен во внутренний чат проверяющий
                    'is_added_one_to_one_master', # добавлен один к одному проверяющий????
                    'any_comment' #есть ли комменты
                   ]
num_need_columns = ['overall_worklogs']
text_need_columns = ['summary'  # описание задачи
                    ]
# Достойны Упоминания
# date : date
# text : списки комментов

In [60]:
df_train['noise_bin'] = np.random.randint(low=-1, high=1, size=len(df_train))
#df_train['noise_10000'] = np.random.randint(low=0, high=100000, size=len(df_train))

In [61]:
df_train.isnull().sum()

id_issue                                0
created                                 0
key                                     0
summary                                 0
project_id                              0
assignee_id                             0
creator_id                              0
overall_worklogs                        0
id_employee                             0
active                                  0
full_name                               0
position                             2235
hiring_type                          1959
payment_type                         2060
salary_calculation_type              9376
english_level                        9055
passport                                0
is_nda_signed                           0
is_labor_contract_signed                0
is_added_to_internal_chats              0
is_added_one_to_one                     0
id                                      0
active_master                           0
full_name_master                  

# Строим baseline-модель🛴
По итогу её и оставили, но только со временнем улучшали

In [264]:

X = df_train[cat_need_columns+['created']+['len_comment']][df_train['overall_worklogs'] <= 80_000].fillna(-1)# затруднительно понять какая замена наиболее релевантная (выбрали -1)
#X = df_train[cat_need_columns+['created']].fillna(-1)# затруднительно понять какая замена наиболее релевантная (выбрали -1)

y = df_train[df_train['overall_worklogs'] <= 80_000]['overall_worklogs']
#y = df_train['overall_worklogs'] 
X_test = df_test[cat_need_columns+['created']+['len_comment']].fillna(-1) # затруднительно понять какая замена наиболее релевантная (выбрали -1)


In [265]:
params = {"iterations": 2500, #1000
          "depth": 5,#5
          "loss_function": "RMSE",
          'eval_metric': 'R2',
          'learning_rate': 0.03,#0.05
           #'l2_leaf_reg': 0,
          'custom_metric' : ['MAE','MAPE','RMSE'],
          "verbose": False,
          "use_best_model": True}

cv_dataset = Pool(data=X,
                  label=y, cat_features=cat_need_columns)

scores_third = cv(cv_dataset,
            params,
            #stratified=True,
            return_models=True,
            fold_count=5,
            shuffle="True",
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.1366898406
bestIteration = 2430

Training on fold [1/5]

bestTest = 0.1537146803
bestIteration = 1459

Training on fold [2/5]

bestTest = 0.1815057823
bestIteration = 2442

Training on fold [3/5]

bestTest = 0.1590495358
bestIteration = 2178

Training on fold [4/5]

bestTest = 0.2053313988
bestIteration = 2493



In [268]:
df_test['overall_worklogs'] = scores_third[1][2].predict(X_test)

In [269]:
df_solution = df_test[['id_issue','overall_worklogs']]
df_solution.to_csv('real_try_1_fold-2222.csv', sep=',', encoding='utf-8', index=False, header = ['id','overall_worklogs'])

# Классифицируем выбросы

In [270]:
train_quantilies = list(map(lambda x: df_train['overall_worklogs'].quantile(0.1 * x), range(1,10)))
train_quantilies

[1200.0, 2400.0, 3600.0, 5100.0, 7200.0, 9000.0, 12600.0, 18000.0, 30900.0]

In [271]:
def num_class(number):
    return(np.sum(list(map(lambda x: int(number >= train_quantilies[x]), range(9)))))
df_train['num_class'] = df_train['overall_worklogs'].apply(num_class)
df_train

,id_issue,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,id_employee,active,full_name,position,hiring_type,payment_type,salary_calculation_type,english_level,passport,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,id,active_master,full_name_master,position_master,hiring_type_master,payment_type_master,salary_calculation_type_master,english_level_master,passport_master,is_nda_signed_master,is_labor_contract_signed_master,is_added_to_internal_chats_master,is_added_one_to_one_master,issue_id,text,comment,any_comment,len_comment,translated_summary,translated_comments,noise_bin,num_class,prob_overall_worklogs
0,819952,2019-10-01 05:57:18,SM-10678,"UI тесты по заказу ""Добро КейДжи""",5,93,93,1800,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN,-,0,0,UI tests commissioned by Dobro KG,-,0,1,0.046305
1,819949,2019-10-01 05:59:45,SM-10679,"UI тесты раздела ""Профиль""",5,93,93,7200,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,819949.0,[Приверила и приняла MR\n\n ],Приверила и приняла MR\n\n,1,1,"UI tests of the ""Profile"" section",Trusted and accepted MR,0,5,0.041633
2,819947,2019-10-01 06:00:38,SM-10680,"UI тесты раздела ""Личный счет""",5,93,93,14400,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN,-,0,0,"UI tests of the section ""Personal account""",,0,7,0.042995
3,819943,2019-10-01 06:02:49,SM-10682,"UI тесты раздела ""Новости""",5,93,93,900,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN,-,0,0,"UI tests of the ""News"" section",,0,0,0.044732
4,819941,2019-10-01 06:03:26,SM-10683,"UI тесты раздела ""Зоны скидок и доплат""",5,93,93,900,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,819941.0,[Приняла mR],Приняла mR,1,1,"UI tests of the ""Zones of discounts and surcha...",-,-1,0,0.050950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9584,688489,2020-01-08 13:42:46,XXO-191,Created Design for a additional content webpages,33,246,246,14400,246,1,Jacklyn Cullum,Web дизайнер,staff,hourly,gross,NaN,1,1,1,1,1,246,1,Jacklyn Cullum,Web дизайнер,staff,hourly,gross,NaN,1,1,1,1,1,688489.0,[clients are happy with a design],clients are happy with a design,1,1,Created Design for a additional content webpages,,-1,7,0.174022
9585,670934,2020-06-02 10:08:14,LJ-1,Set up the neo4j by using a dataset from Slack,56,2,2,16200,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,670934.0,[I have read the neo4j documentation and also ...,I have read the neo4j documentation and also I...,1,3,Set up the neo4j by using a dataset from Slack,,-1,7,0.138012
9586,670932,2020-06-05 09:52:12,LJ-3,Set up a graph in neo4j by GitLab dataset,56,2,2,3600,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,NaN,NaN,-,0,0,Set up a graph in neo4j by GitLab dataset,example:,-1,3,0.085661
9587,670931,2020-06-05 13:46:38,LJ-4,Parse a dataset from teacherly slack,56,2,2,6300,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,NaN,NaN,-,0,0,Parse a dataset from teacherly slack,,-1,4,0.099294


In [272]:
def num_class(number):
    return(np.sum(list(map(lambda x: int(number >= train_quantilies[x]), [8]))))
df_train['num_class'] = df_train['overall_worklogs'].apply(num_class)
df_train

,id_issue,created,key,summary,project_id,assignee_id,creator_id,overall_worklogs,id_employee,active,full_name,position,hiring_type,payment_type,salary_calculation_type,english_level,passport,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,id,active_master,full_name_master,position_master,hiring_type_master,payment_type_master,salary_calculation_type_master,english_level_master,passport_master,is_nda_signed_master,is_labor_contract_signed_master,is_added_to_internal_chats_master,is_added_one_to_one_master,issue_id,text,comment,any_comment,len_comment,translated_summary,translated_comments,noise_bin,num_class,prob_overall_worklogs
0,819952,2019-10-01 05:57:18,SM-10678,"UI тесты по заказу ""Добро КейДжи""",5,93,93,1800,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN,-,0,0,UI tests commissioned by Dobro KG,-,0,0,0.046305
1,819949,2019-10-01 05:59:45,SM-10679,"UI тесты раздела ""Профиль""",5,93,93,7200,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,819949.0,[Приверила и приняла MR\n\n ],Приверила и приняла MR\n\n,1,1,"UI tests of the ""Profile"" section",Trusted and accepted MR,0,0,0.041633
2,819947,2019-10-01 06:00:38,SM-10680,"UI тесты раздела ""Личный счет""",5,93,93,14400,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN,-,0,0,"UI tests of the section ""Personal account""",,0,0,0.042995
3,819943,2019-10-01 06:02:49,SM-10682,"UI тесты раздела ""Новости""",5,93,93,900,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,NaN,NaN,-,0,0,"UI tests of the ""News"" section",,0,0,0.044732
4,819941,2019-10-01 06:03:26,SM-10683,"UI тесты раздела ""Зоны скидок и доплат""",5,93,93,900,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,93,1,Olga Bush,Разработчик мобильных приложений,staff,fixed,NaN,NaN,1,1,1,1,1,819941.0,[Приняла mR],Приняла mR,1,1,"UI tests of the ""Zones of discounts and surcha...",-,-1,0,0.050950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9584,688489,2020-01-08 13:42:46,XXO-191,Created Design for a additional content webpages,33,246,246,14400,246,1,Jacklyn Cullum,Web дизайнер,staff,hourly,gross,NaN,1,1,1,1,1,246,1,Jacklyn Cullum,Web дизайнер,staff,hourly,gross,NaN,1,1,1,1,1,688489.0,[clients are happy with a design],clients are happy with a design,1,1,Created Design for a additional content webpages,,-1,0,0.174022
9585,670934,2020-06-02 10:08:14,LJ-1,Set up the neo4j by using a dataset from Slack,56,2,2,16200,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,670934.0,[I have read the neo4j documentation and also ...,I have read the neo4j documentation and also I...,1,3,Set up the neo4j by using a dataset from Slack,,-1,0,0.138012
9586,670932,2020-06-05 09:52:12,LJ-3,Set up a graph in neo4j by GitLab dataset,56,2,2,3600,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,NaN,NaN,-,0,0,Set up a graph in neo4j by GitLab dataset,example:,-1,0,0.085661
9587,670931,2020-06-05 13:46:38,LJ-4,Parse a dataset from teacherly slack,56,2,2,6300,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,2,0,Dan Guerra,Web-разработчик,staff,fixed,NaN,NaN,0,0,0,0,0,NaN,NaN,-,0,0,Parse a dataset from teacherly slack,,-1,0,0.099294


In [273]:

X = df_train[cat_need_columns+['created']+['len_comment'] + ['translated_summary']+['translated_comments']].fillna(-1)# затруднительно понять какая замена наиболее релевантная (выбрали -1)
y = df_train['num_class']

X_test = df_test[cat_need_columns+['created']+['len_comment'] + ['translated_summary']+['translated_comments']].fillna(-1) # затруднительно понять какая замена наиболее релевантная (выбрали -1)


In [274]:
params = {"iterations": 1000,
          "depth": 5,
          "loss_function": "Logloss",
          'eval_metric': 'MCC',
          'learning_rate': 0.05,
           #'l2_leaf_reg': 0,
          'custom_metric' : ['F1','Precision'],
          "verbose": False,
          "use_best_model": True}

cv_dataset = Pool(data=X,
                  label=y, cat_features=cat_need_columns,
                 text_features = ['translated_summary','translated_comments' ])

scores_classification = cv(cv_dataset,
            params,
            stratified=True,
            return_models=True,
            fold_count=5,
            shuffle="True",
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.3228764231
bestIteration = 786

Training on fold [1/5]

bestTest = 0.3459899823
bestIteration = 478

Training on fold [2/5]

bestTest = 0.2696921859
bestIteration = 810

Training on fold [3/5]

bestTest = 0.2676426952
bestIteration = 921

Training on fold [4/5]

bestTest = 0.2642018053
bestIteration = 968



In [275]:
df_test['newest_overall_worklogs'] = scores_classification[1][0].predict(X_test, prediction_type='Class')
df_test['prob_overall_worklogs'] = scores_classification[1][0].predict(X_test, prediction_type='Probability')[:,1]
df_test


,id_issue,created,key,summary,project_id,assignee_id,creator_id,id_employee,active,full_name,position,hiring_type,payment_type,salary_calculation_type,english_level,passport,is_nda_signed,is_labor_contract_signed,is_added_to_internal_chats,is_added_one_to_one,id,active_master,full_name_master,position_master,hiring_type_master,payment_type_master,salary_calculation_type_master,english_level_master,passport_master,is_nda_signed_master,is_labor_contract_signed_master,is_added_to_internal_chats_master,is_added_one_to_one_master,issue_id,text,comment,any_comment,len_comment,translated_summary,translated_comments,overall_worklogs,newest_overall_worklogs,prob_overall_worklogs
0,675975,2020-01-15 09:52:57.000,SM-11140,Выгрузить все mp3 файлы звонков на диск,5,112,112,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,675975.0,[Ресерч по rsync:\n\n[https://losst.ru/rsync-p...,Ресерч по rsync:\n\n[https://losst.ru/rsync-pr...,1,8,Download all mp3 files of calls to disk,Research on rsync:,13873.452520,0,0.333835
1,675972,2020-01-15 09:54:35.000,SM-11141,Удалить файлы mp3 звонков с сервера до 01.01.2019,5,112,112,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,675972.0,[Задача приостановлена в связи с заморозкой ра...,Задача приостановлена в связи с заморозкой раб...,1,4,Delete mp3 call files from the server before 0...,,22613.018883,0,0.110307
2,675857,2020-05-04 09:47:24.000,SM-11186,настройка DNS на cloudflare,5,112,112,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,675857.0,[Создал учетку в клауд флейр для дальнешего пе...,Создал учетку в клауд флейр для дальнешего пер...,1,3,configuring DNS on cloudflare,[https://losst.ru/rsync-primery-synchronizatsii],11719.382862,0,0.087837
3,675849,2020-05-12 08:53:22.000,SM-11189,Выяснить действительно ли причина в поломанной...,5,112,112,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,675849.0,[Посмотрел видео по репликации\r\n\r\n[https:/...,Посмотрел видео по репликации\r\n\r\n[https://...,1,4,Find out if the reason is broken replication o...,,23976.995790,0,0.123370
4,675845,2020-05-13 12:02:28.000,SM-11191,Разобраться с проблемой репликации,5,112,112,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,112,0,Latoya Daughtrey,DevOps инженер,staff,fixed,NaN,NaN,0,0,0,0,0,675845.0,[Прочитал доку [https://docs.google.com/docume...,Прочитал доку [https://docs.google.com/documen...,1,6,Deal with the replication problem,[https://andreyex.ru/linux/komandy-linux-i-kom...,17498.828887,0,0.253902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1065,1990701,2021-06-22 17:23:10.572,FPY-71,Добавить базовую аутентификацию для АПИ к камерам,29,191,191,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,NaN,NaN,-,0,0,Add Basic Authentication for API to Cameras,,11125.957391,0,0.058742
1066,2105184,2021-07-02 15:00:08.924,FPY-79,[back] Настроить отправку писем после регистрации,29,191,191,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,NaN,NaN,-,0,0,[back] Set up sending emails after registration,,11125.957391,0,0.062043
1067,2170252,2021-07-08 13:05:34.082,FPY-82,Обновлять статус камеры на неактивный при отсу...,29,191,191,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,191,1,Kirk Hines,Web-разработчик,staff,hourly,NaN,C1,1,1,1,1,1,2170252.0,[В настройки проекта в constance вынес конфиг ...,В настройки проекта в constance вынес конфиг {...,1,2,Update camera st

In [276]:
alpha = 0.65
print(df_train[['overall_worklogs','num_class','prob_overall_worklogs']][df_train['prob_overall_worklogs'] > alpha]['num_class'].sum())
print(df_train[['overall_worklogs','num_class','prob_overall_worklogs']][df_train['prob_overall_worklogs'] > alpha].shape)
df_train[['overall_worklogs','num_class','prob_overall_worklogs']][df_train['prob_overall_worklogs'] > alpha][df_train['num_class'] == 0]

112
(115, 3)


C:\Users\MgDuck\AppData\Local\Temp\ipykernel_10884\2654710468.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_train[['overall_worklogs','num_class','prob_overall_worklogs']][df_train['prob_overall_worklogs'] > alpha][df_train['num_class'] == 0]


,overall_worklogs,num_class,prob_overall_worklogs
9486,18000,0,0.895206
9551,23400,0,0.679429
9552,27900,0,0.672599


In [277]:
df_train[df_train['overall_worklogs']>=30900]['overall_worklogs'].median()

50100.0

In [278]:
for j in range(5):
    df_test['prob_overall_worklogs'] = scores_classification[1][j].predict(X_test, prediction_type='Probability')[:,1]
    for i in df_test[['overall_worklogs','prob_overall_worklogs']][df_test['prob_overall_worklogs'] > 0.65].index:
        df_test['overall_worklogs'][i] = max(40100.0,df_test['overall_worklogs'][i]) #было 30900

C:\Users\MgDuck\AppData\Local\Temp\ipykernel_10884\3865200418.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['overall_worklogs'][i] = max(40100.0,df_test['overall_worklogs'][i]) #было 30900
C:\Users\MgDuck\AppData\Local\Temp\ipykernel_10884\3865200418.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['overall_worklogs'][i] = max(40100.0,df_test['overall_worklogs'][i]) #было 30900
C:\Users\MgDuck\AppData\Local\Temp\ipykernel_10884\3865200418.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.

In [279]:
df_solution = df_test[['id_issue','overall_worklogs']]
df_solution.to_csv('mb_final.csv', sep=',', encoding='utf-8', index=False, header = ['id','overall_worklogs'])